# Movie Queries Answers  + Extras
***

## Schema  
  
#### Movie (mID, title, year, director)  
#### Reviewer (rID, name)  
#### Rating (rID, mID, stars, ratingDate)

*** **** *** **** ************************************

#### Q1: Find the titles of all movies directed by Steven Spielberg.

#### Q2: Find all years that have a movie that received a rating of 4 or 5, and sort them in increasing order.

#### Q3: Find the titles of all movies that have no ratings.

#### Q4: Some reviewers didn't provide a date with their rating. Find the names of all reviewers who have ratings with a NULL value for the date.

#### Q5: Write a query to return the ratings data in a more readable format: reviewer name, movie title, stars, and ratingDate. Also, sort the data, first by reviewer name, then by movie title, and lastly by number of stars.

#### Q6: For all cases where the same reviewer rated the same movie twice and gave it a higher rating the second time, return the reviewer's name and the title of the movie.

#### Q7: For each movie that has at least one rating, find the highest number of stars that movie received. Return the movie title and number of stars. Sort by movie title.

#### Q8: For each movie, return the title and the 'rating spread', that is, the difference between highest and lowerst ratings given to that movie. Sort by rating spread from highest to lowest, then by movie title.

#### Q9: Find the difference between the average rating of movies released before 1980 and the average rating of movies released after 1980. (Make sure to calculate the average rating for each movie, then the average of those averages for movies before 1980 and movies after. Don't just calculate the overall average rating before and after 1980.)

## Extras  
***

#### Q1: Find the names of all reviewers who rated Gone with the Wind.

In [ ]:
SELECT v.name  
FROM Reviewer v  
WHERE v.rID IN (SELECT r.rID  
                FROM Rating r  
                JOIN Movie m ON r.mID = m.mID  
                WHERE m.title = 'Gone with the Wind');

#### Q2: For any rating where the reviewer is the same as the director of the movie, return the reviewer name, movie title, and number of stars.

In [ ]:
SELECT v.name, m.title, r.stars  
FROM Movie m  
JOIN Rating r ON m.mID = r.mID  
JOIN Reviewer v ON r.rID = v.rID  
WHERE v.name = m.director;  

#### Q3: Return all reviewer names and movie names together in a single list, alphabetized. 

In [ ]:
SELECT name  
FROM Reviewer v  
UNION  
SELECT title  
FROM Movie m  
ORDER BY v.name;

#### Q4: Find the titles of all movies not reviewed by Chris Jackson.

In [ ]:
SELECT m.title  
FROM Movie m  
WHERE m.mID NOT IN (SELECT r.mID  
                    FROM Rating r INNER JOIN Reviewer v ON r.rID = v.rID  
                    WHERE v.name = 'Chris Jackson');  

#### Q5: For all pairs of reviewers such that both reviewers gave a rating to the same movie, return the names of both reviewers. Eliminate duplicates, don't pair reviewers with themselves, and include each pair only once. For each pair, return the names in the pair in alphabetical order. 

In [ ]:
SELECT DISTINCT r1.name, r2.name  
FROM (Reviewer NATURAL JOIN Rating) r1, (Reviewer NATURAL JOIN Rating) r2  
WHERE r1.mID = r2.mID AND r1.name < r2.name  
ORDER BY r1.name, r2.name;

#### Q6: For each rating that is the lowest (fewest stars) currently in the database, return the reviewer name, movie title, and number of stars.

In [ ]:
SELECT v.name, m.title, r.stars  
FROM Movie m  
JOIN Rating r ON m.mID = r.mID  
JOIN Reviewer v ON r.rID = v.rID  
WHERE r.stars <= (SELECT MIN(stars)  
                  FROM Rating);

#### Q7: List movie titles and average ratings, from highest-rated to lowest-rated. If two or more movies have the same average rating, list them in alphabetical order. 

In [ ]:
SELECT m.title, AVG(r.stars) AS average_rating
FROM Movie m
JOIN Rating r ON m.mID = r.mID
GROUP BY m.title
ORDER BY average_rating DESC;

#### Q8: Find the names of all reviewers who have contributed three or more ratings. 

In [ ]:
SELECT v.name
FROM Reviewer v
JOIN Rating r ON v.rID = r.rID
GROUP BY v.name
HAVING COUNT(v.name) >=3;

#### Q9: Some directors directed more than one movie. For all such directors, return the titles of all movies directed by them, along with director name. Sort by director name, then movie title. 

In [ ]:
SELECT m.title, m.director
FROM Movie m
WHERE m.director IN (SELECT m1.director
                     FROM Movie m1                     
                     GROUP BY m1.director
                     HAVING COUNT(m1.director) > 1)
ORDER BY m.director, m.title;

#### Q10: Find the movie(s) with the highest average rating. Return the movie title(s) and average rating. 

In [ ]:
SELECT m.title, AVG(r.stars) AS avg
FROM Movie m
JOIN Rating r ON m.mID = r.mID
GROUP BY r.mID
HAVING AVG(r.stars) = (SELECT AVG(r1.stars)
                       FROM Movie m1 JOIN Rating r1 ON m1.mID = r1.mID
                       GROUP BY r1.mID 
                       ORDER BY AVG(r1.stars) DESC
                       LIMIT 1);

#### Q11: Find the movie(s) with the lowest average rating. Return the movie title(s) and average rating.

In [ ]:
SELECT m.title, AVG(r.stars) AS avg
FROM Movie m
JOIN Rating r ON m.mID = r.mID
GROUP BY r.mID
HAVING AVG(r.stars) = (SELECT AVG(r1.stars)
                       FROM Movie m1 JOIN Rating r1 ON m1.mID = r1.mID
                       GROUP BY r1.mID
                       ORDER BY AVG(r1.stars)
                       LIMIT 1);

#### Q12: For each director, return the director's name together with the title(s) of the movie(s) they directed that received the highest rating among all their movies, and the value of that rating. Ignore movies whose director is NULL.

In [ ]:
SELECT m.director, m.title, MAX(r.stars)
FROM Movie m
JOIN Rating r ON m.mID = r.mID
WHERE m.director IS NOT NULL
GROUP BY m.director;